In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback
from peft import LoraConfig, TaskType, get_peft_model
import torch
import psutil
import numpy as np
import evaluate

2025-05-14 09:47:50.918060: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-14 09:47:50.924562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747198070.932713 1023751 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747198070.935143 1023751 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-14 09:47:50.943890: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
torch.cuda.set_per_process_memory_fraction(1.0, 0)  # Use maximum available memory
torch.cuda.memory_max_split_size_mb = 64  # Set the max split size to avoid fragmentation

In [3]:
def print_memory_footprint():
    # GPU memory usage
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert to GB
        gpu_memory_cached = torch.cuda.memory_reserved() / (1024 ** 3)  # Cached memory
        print(f"[GPU] Memory Allocated: {gpu_memory:.2f} GB, Cached: {gpu_memory_cached:.2f} GB")
    else:
        print("[GPU] No GPU detected.")

    # CPU memory usage
    memory = psutil.virtual_memory()
    used_memory_gb = memory.used / (1024 ** 3)  # Convert to GB
    total_memory_gb = memory.total / (1024 ** 3)
    print(f"[CPU] Memory Usage: {used_memory_gb:.2f} GB / {total_memory_gb:.2f} GB")

In [4]:
'''
print("First example of blended_skill_talk training set:")
print(dataset['train'][0])
'''

'\nprint("First example of blended_skill_talk training set:")\nprint(dataset[\'train\'][0])\n'

In [ ]:
# Load dataset and tokenizer
# dataset = load_dataset("daily_dialog")

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token, so we use eos_token


def tokenize_function(examples):
    # Concatenate dialog turns into a single string for language modeling
    texts = [" ".join(dialog) for dialog in examples["dialog"]]
    return tokenizer(texts, truncation=True, max_length=512)

# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["dialog", "act", "emotion"])
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

In [6]:
dataset["train"][0:1]

{'dialog': [['Say , Jim , how about going for a few beers after dinner ? ',
   ' You know that is tempting but is really not good for our fitness . ',
   ' What do you mean ? It will help us to relax . ',
   " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
   " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
   ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
   " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
   ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
   " Good.Let ' s go now . ",
   ' All right . ']],
 'act': [[3, 4, 2, 2, 2, 3, 4, 1, 3, 4]],
 'emotion': [[0, 0, 0, 0, 0, 0, 4, 4, 4, 4]]}

In [7]:
small_train_dataset[0:1]

{'input_ids': [[1,
   7146,
   591,
   29915,
   276,
   373,
   7613,
   1738,
   259,
   4874,
   869,
   739,
   471,
   577,
   11660,
   7176,
   869,
   306,
   29915,
   29885,
   28043,
   29899,
   449,
   869,
   2803,
   29915,
   29879,
   1284,
   1749,
   22091,
   869,
   259,
   526,
   896,
   3474,
   22091,
   470,
   263,
   275,
   280,
   22091,
   1577,
   259,
   1235,
   592,
   1074,
   2023,
   4874,
   1919,
   697,
   3474,
   12949,
   322,
   697,
   263,
   275,
   280,
   12949,
   869,
   259,
   3431,
   869,
   1205,
   508,
   306,
   11302,
   590,
   12949,
   411,
   366,
   1577,
   306,
   5821,
   278,
   697,
   2978,
   278,
   3474,
   869,
   306,
   29915,
   29885,
   263,
   16403,
   11340,
   261,
   869,
   306,
   2337,
   679,
   4799,
   29879,
   860,
   322,
   508,
   2360,
   26681,
   2745,
   1156,
   306,
   29915,
   345,
   2982,
   287,
   869,
   259,
   393,
   29915,
   29879,
   2691,
   869,
   306,
   29915,
   298

In [8]:
'''
processed_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['dialog', 'act', 'emotion'])
small_train_dataset = processed_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = processed_datasets["validation"].shuffle(seed=42).select(range(100)) # Using validation for a small eval set

print_memory_footprint()
'''

'\nprocessed_datasets = dataset.map(preprocess_function, batched=True, remove_columns=[\'dialog\', \'act\', \'emotion\'])\nsmall_train_dataset = processed_datasets["train"].shuffle(seed=42).select(range(1000))\nsmall_eval_dataset = processed_datasets["validation"].shuffle(seed=42).select(range(100)) # Using validation for a small eval set\n\nprint_memory_footprint()\n'

In [9]:
# LoRA configuration for causal language modeling
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "k_proj"],
)

# %%

In [10]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained GPT-2 language model
# Load pre-trained GPT-2 model
#model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                                             torch_dtype=torch.bfloat16,).to(device)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Enable gradient checkpointing if you run into memory issues
#model.gradient_checkpointing_enable()

# %%
# Print the model's architecture to inspect the names of the modules
#print(model)

# %%
print_memory_footprint()

Using device: cuda
[GPU] Memory Allocated: 2.06 GB, Cached: 2.14 GB
[CPU] Memory Usage: 4.58 GB / 62.56 GB


In [11]:
# %%

# Data collator for language modeling (masks tokens for prediction)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # mlm=False for causal LM

# Perplexity as metric
#perplexity_metric = evaluate.load("perplexity", module_type="metric")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    shift_logits = torch.tensor(logits[:, :-1, :])
    shift_labels = torch.tensor(labels[:, 1:])
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    ppl = torch.exp(loss).item()
    return {"perplexity": ppl}

# Training args
training_args = TrainingArguments(
    output_dir="./gpt2-dialog-finetuned",
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=250,
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,
    bf16=True,  # Enable bfloat16
    fp16=False,  # Disable fp16 to avoid conflicts
)
# %%
print_memory_footprint()

[GPU] Memory Allocated: 2.06 GB, Cached: 2.14 GB
[CPU] Memory Usage: 4.58 GB / 62.56 GB


In [12]:
class MemoryCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        print("\nMemory footprint after evaluation:")
        print_memory_footprint()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    #eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    callbacks=[MemoryCallback()]
)


/tmp/ipykernel_1023751/3187942852.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:

print("--- GPT-2 Before Fine-tuning ---")
model_before_finetune = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
             torch_dtype=torch.bfloat16).to(device)

input_text_before = "Hello, how are you?"
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- GPT-2 Before Fine-tuning ---
Input: Hello, how are you?
Generated Response (Before): Hello, how are you?
I am fine, thank you.
How are you? I am fine, thank you.
Can you repeat that again?
Can you repeat that again, please?
Can you repeat that again,
---


In [23]:

print("--- TinyLlama Before Fine-tuning ---")
#model_before_finetune = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

input_text_before = "The USA celebrate independence day on "
input_ids_before = tokenizer.encode(input_text_before, return_tensors="pt").to(device)

print(f"Input: {input_text_before}")
output_before = model_before_finetune.generate(
    input_ids_before,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)
generated_response_before = tokenizer.decode(output_before[0], skip_special_tokens=True)
print(f"Generated Response (Before): {generated_response_before}")
print("---")


--- TinyLlama Before Fine-tuning ---
Input: The USA celebrate independence day on 
Generated Response (Before): The USA celebrate independence day on 4th July.

2. The USA celebrate independence day on 4th July.

3. The USA celebrate independence day on 4th July.

4. The
---


In [15]:
any(p.requires_grad for p in model.parameters())

True

In [16]:
trainer.train()

Step,Training Loss
250,2.264800
500,2.132000
750,2.101500
1000,2.103500
1250,2.109500
1500,2.087300
1750,2.109500
2000,2.073900
2250,2.077000
2500,2.079400


TrainOutput(global_step=16670, training_loss=2.068904502790848, metrics={'train_runtime': 2134.2274, 'train_samples_per_second': 46.855, 'train_steps_per_second': 7.811, 'total_flos': 1.6003618348577587e+17, 'train_loss': 2.068904502790848, 'epoch': 10.0})

In [17]:
trainer.save_model('TinyLlama-dailydialog-1000-May25')

In [18]:
# %%
print_memory_footprint()

[GPU] Memory Allocated: 2.08 GB, Cached: 13.16 GB
[CPU] Memory Usage: 5.12 GB / 62.56 GB


In [19]:
# For generation, we'll load the model and use the `generate` method
model_for_generation = AutoModelForCausalLM.from_pretrained('TinyLlama-dailydialog-1000-May25').to(device)
#model_for_generation = get_peft_model(model_for_generation, lora_config) # Ensure LoRA is applied

input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")

print_memory_footprint()

/home/sajjad/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Generating response:
Input: Hello, how are you?
Generated Response: Hello, how are you?   I'm fine.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?   I'm fine too.How about you?  
[GPU] Memory Allocated: 6.19 GB, Cached: 13.16 GB
[CPU] Memory Usage: 5.90 GB / 62.56 GB


In [20]:
input_text = "The USA celebrate independence day on "
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

print("\nGenerating response:")
output = model_for_generation.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Input: {input_text}")
print(f"Generated Response: {generated_response}")


Generating response:
Input: The USA celebrate independence day on 
Generated Response: The USA celebrate independence day on 4th July .   What do you think about it ?   I think it's a good idea .   Why ?   Because it's a good day to celebrate .   I agree .   What do you think about the flag ?   I think it's a good flag .   What do you think about the national anthem ?   I think it's a good anthem .   What do you think about the national holiday ?   I think it's a good holiday .   What do you think about the people ?   I think they are very friendly .   What do you think about the government ?   I think they are very good .   What do
